In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [125]:
# Load individual CSV files
orders_df = pd.read_csv(r"C:\Users\hp\Downloads\Pizza+Place+Sales\pizza_sales\orders.csv")
order_details_df = pd.read_csv(r"C:\Users\hp\Downloads\Pizza+Place+Sales\pizza_sales\order_details.csv") 
pizzas_df = pd.read_csv(r"C:\Users\hp\Downloads\Pizza+Place+Sales\pizza_sales\pizzas.csv")
pizza_types_df = pd.read_csv(r"C:\Users\hp\Downloads\Pizza+Place+Sales\pizza_sales\pizza_types.csv", encoding='latin-1')
   

In [126]:
# Convert date and time columns
orders_df['date'] = pd.to_datetime(orders_df['date'])
# Convert to proper time objects
orders_df['time'] = pd.to_datetime(orders_df['time'], format='%H:%M:%S').dt.time

In [127]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21350 entries, 0 to 21349
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   order_id  21350 non-null  int64         
 1   date      21350 non-null  datetime64[ns]
 2   time      21350 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 500.5+ KB


In [128]:
order_details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   order_details_id  48620 non-null  int64 
 1   order_id          48620 non-null  int64 
 2   pizza_id          48620 non-null  object
 3   quantity          48620 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [129]:
order_details_df.isna().sum()

order_details_id    0
order_id            0
pizza_id            0
quantity            0
dtype: int64

In [130]:
orders_df.isna().sum()

order_id    0
date        0
time        0
dtype: int64

In [131]:
pizzas_df.isna().sum()

pizza_id         0
pizza_type_id    0
size             0
price            0
dtype: int64

In [132]:
pizza_types_df.isna().sum()

pizza_type_id    0
name             0
category         0
ingredients      0
dtype: int64

In [133]:
 #Join all datasets into a single DataFrame
print(" Joining datasets")

# Start with order_details and join with orders
merged_df = order_details_df.merge(orders_df, on='order_id', how='left')

# Join with pizzas to get price information
merged_df = merged_df.merge(pizzas_df, on='pizza_id', how='left')

# Join with pizza_types to get pizza names and details
merged_df = merged_df.merge(pizza_types_df, on='pizza_type_id', how='left')

merged_df.head()

 Joining datasets


,order_details_id,order_id,pizza_id,quantity,date,time,pizza_type_id,size,price,name,category,ingredients
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,classic_dlx,M,16.00,The Classic Deluxe Pizza,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,five_cheese,L,18.50,The Five Cheese Pizza,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go..."
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,ital_supr,L,20.75,The Italian Supreme Pizza,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni..."
4,5,2,mexicana_m,1,2015-01-01,11:57:40,mexicana,M,16.00,The Mexicana Pizza,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O..."


In [134]:
merged_df.isna().sum()

order_details_id    0
order_id            0
pizza_id            0
quantity            0
date                0
time                0
pizza_type_id       0
size                0
price               0
name                0
category            0
ingredients         0
dtype: int64

In [135]:
#1 total revenue
total_revenue = merged_df['price'].sum()
total_revenue

np.float64(801944.7000000001)

In [136]:
#2 total quantity sold
total_quantity = merged_df['quantity'].sum()
total_quantity

np.int64(49574)

In [137]:
#3 total Orders
total_orders = merged_df['order_id'].nunique()
total_orders

21350

In [138]:
#4 number of pizza types
pizza_types_count = merged_df['pizza_type_id'].nunique()
pizza_types_count 

32

In [139]:
#5 average price for pizza
avg_price = merged_df['price'].mean()
avg_price 

np.float64(16.494132044426163)

In [140]:
#7
merged_df['day_of_week'] = merged_df['date'].dt.day_name()
merged_df['day_num'] = merged_df['date'].dt.dayofweek

daily_sales = merged_df.groupby(['day_num', 'day_of_week']).agg({
    'price': 'sum',
    'quantity': 'sum', 
    'order_id': 'nunique'
}).reset_index()

daily_sales = daily_sales.sort_values('price', ascending=False)

print("   Daily Sales Ranking:")
for i, row in daily_sales.iterrows():
    print(f"   • {row['day_of_week']:9s}: ${row['price']:7,.0f} ({row['order_id']:3d} orders)")

best_day = daily_sales.iloc[0]['day_of_week']
best_day

   Daily Sales Ranking:
   • Friday   : $133,774 (3538 orders)
   • Thursday : $120,912 (3239 orders)
   • Saturday : $120,865 (3158 orders)
   • Wednesday: $112,006 (3024 orders)
   • Tuesday  : $111,765 (2973 orders)
   • Monday   : $105,357 (2794 orders)
   • Sunday   : $ 97,265 (2624 orders)


'Friday'

In [141]:
#6 peak hours of sales
merged_df['hour'] = pd.to_datetime(merged_df['time'], format='%H:%M:%S').dt.hour
hourly_sales = merged_df.groupby('hour')['price'].sum().reset_index()

peak_hour_sales = hourly_sales.loc[hourly_sales['price'].idxmax(), 'hour']
print(f" Peak sales hour: {peak_hour_sales}:00 (${hourly_sales['price'].max():,.2f})")

 Peak sales hour: 12:00 ($108,047.75)


In [142]:
#9 sales made in each month
merged_df['month'] = merged_df['date'].dt.month
merged_df['month_name'] = merged_df['date'].dt.month_name()
monthly_sales = merged_df.groupby(['month', 'month_name'])['price'].sum().reset_index()
monthly_sales = monthly_sales.sort_values('month')
print("   Monthly Sales:")
for _, row in monthly_sales.iterrows():
    
    print(f" {row['month_name']}: ${row['price']:,.2f}")

# Identify trend
max_month = monthly_sales.loc[monthly_sales['price'].idxmax(), 'month_name']
min_month = monthly_sales.loc[monthly_sales['price'].idxmin(), 'month_name']
print(f"\n  Highest sales: {max_month}")
print(f"\n  Lowest sales: {min_month}")


   Monthly Sales:
 January: $68,472.70
 February: $64,067.40
 March: $69,198.00
 April: $67,286.20
 May: $69,939.35
 June: $66,796.30
 July: $71,027.45
 August: $67,068.80
 September: $63,018.00
 October: $62,566.50
 November: $69,054.00
 December: $63,450.00

  Highest sales: July

  Lowest sales: October


In [143]:
#10
pizza_performance = merged_df.groupby('name').agg({
    'quantity': 'sum',
    'price': 'sum',
    'order_id': 'nunique'
}).reset_index()

pizza_performance.columns = ['name', 'total_quantity', 'total_revenue', 'unique_orders']
pizza_performance['avg_price'] = pizza_performance['total_revenue'] / pizza_performance['total_quantity']

# Sort by quantity (performance metr
pizza_performance_sorted = pizza_performance.sort_values('total_quantity')

print("Lowest Quantities Sold")
bottom_5 = pizza_performance_sorted.head()
bottom_5 


Lowest Quantities Sold


,name,total_quantity,total_revenue,unique_orders,avg_price
2,The Brie Carre Pizza,490,11352.00,480,23.167347
16,The Mediterranean Pizza,934,15163.00,912,16.234475
3,The Calabrese Pizza,937,15763.75,918,16.823639
28,The Spinach Supreme Pizza,950,15124.00,918,15.920000
24,The Soppressata Pizza,961,16342.75,941,17.005983
